In [1]:
import pandas as pd
import numpy as np
import knnimpute as knnImp
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import KMeans

In [2]:
caseDf = pd.read_csv('case.csv',low_memory=False)
ctrlDf = pd.read_csv('ctrl.csv',low_memory=False)

patientLabels = ctrlDf['Arrays']
patientLabels = patientLabels.append(caseDf['Arrays']).reset_index(drop=True)

In [3]:
#imputes median value from 7-nn for missing values ctrl group
ctrlData = ctrlDf.drop('Arrays',axis=1)
features = ctrlData.columns
ctrlDataImputed = knnImp.knn_impute_few_observed(ctrlData.as_matrix(),ctrlData.isnull().as_matrix(),7)
ctrlDf = pd.DataFrame(data = ctrlDataImputed, columns = features)

In [4]:
#imputes median value from 7-nn for missing values case group
caseData = caseDf.drop('Arrays',axis=1)
features = caseData.columns
caseDataImputed = knnImp.knn_impute_few_observed(caseData.as_matrix(),caseData.isnull().as_matrix(),7)
caseDf =pd.DataFrame(data = caseDataImputed, columns = features)

In [5]:
allPatientsKnnImputedData = pd.concat([ctrlDf,caseDf])
#allPatientsKnnImputedData.to_csv('/Users/Daniel/Documents/Data Mining 514/HW3/allPatientsKnnImputedData.csv',low_memory=False)

In [6]:
#gets data and sets up Patients and Normal indices, X and Y for estimator and validation
allPatientsKnnImputedData = pd.read_csv('/Users/Daniel/Documents/Data Mining 514/HW3/allPatientsKnnImputedData.csv',low_memory=False)
P = np.linspace(0,175,num=176)#indices of normal ctrl
N = np.linspace(176,363,num=188)#indices of case patients
Y = allPatientsKnnImputedData['Classes']
X = allPatientsKnnImputedData.drop('Classes',axis=1)

In [7]:
#fits estimator bottom up for 2 clusters
estimator = AgglomerativeClustering(n_clusters=2, affinity='cosine', linkage = 'complete')
estimator.fit(X.sub(X.mean(axis=0), axis=1))
bottomUp2ClusterLabels = estimator.labels_

#determines the ctrl and case labels
results = abs((estimator.labels_+1)- Y.as_matrix())
if sum(results)/len(results) > .5:
    misclassifiedBottomUp = np.nonzero(results-1)
    pB = np.where(estimator.labels_ ==0)
    nB = np.where(estimator.labels_ ==1)
else:
    misclassifiedBottomUp = np.nonzero((results))
    pB = np.where(estimator.labels_ ==1)
    nB = np.where(estimator.labels_ ==0)
max(sum(abs((estimator.labels_+1)- Y.as_matrix()))/len(Y),1-sum(abs((estimator.labels_+1)- Y.as_matrix()))/len(Y))

0.59340659340659341

In [ ]:
TPR = len(np.intersect1d(pB[0][pB[0]<176],P))/len(P)
FPR = 1 - len(np.intersect1d(pB[0][pB[0]<176],P))/len(P)
print("The true Positie rate is %f and the False Positive rate it %f"%(TPR,FPR) )
print("The size of the clusters are %d for the positve group and %d for the negative group"%(len(pB[0]),len(nB[0])))

In [ ]:
patientList = []
for pat in patientLabels[misclassifiedBottomUp[0]]:
    patientList.append(pat)
print(patientList)

In [ ]:
patientList = []
for pat in 'cba':
    patientList.append(pat)
    print(pat)
print(patientList)

In [ ]:
#Bottom up 4 groups clustering
estimator = AgglomerativeClustering(n_clusters=4, affinity='cosine', linkage = 'complete')
estimator.fit(X.sub(X.mean(axis=0), axis=1))
bottomUp4ClusterLabels = estimator.labels_
bottomUp2ndLevelDict = {'group0':[],
                        'group1':[],
                        'group2':[],
                        'group3':[]
                       }
bottomUp2ndLevelStats = {'group0':0,
                        'group1':0,
                        'group2':0,
                        'group3':0
                       }
totalMisclassified4Clusters=0
for label in np.unique(bottomUp4ClusterLabels):
    group = 'group{0}'.format(label)
    bottomUp2ndLevelDict[group] = X.iloc[np.where(bottomUp4ClusterLabels==label)]
    tempAccuracy = sum(bottomUp2ndLevelDict[group].index<176)/len(bottomUp2ndLevelDict[group])
    if tempAccuracy>.5:
        bottomUp2ndLevelStats[group] = [len(bottomUp2ndLevelDict[group]),tempAccuracy]
    else:
        bottomUp2ndLevelStats[group] = [len(bottomUp2ndLevelDict[group]),1-tempAccuracy]
    totalMisclassified4Clusters = totalMisclassified4Clusters  + bottomUp2ndLevelStats[group][0]       


In [ ]:
bottomUp2ndLevelStats

In [ ]:
#Bottom up 8 groups clustering

estimator = AgglomerativeClustering(n_clusters=8, affinity='cosine', linkage = 'average')
estimator.fit(X.sub(X.mean(axis=0), axis=1))
bottomUp8ClusterLabels = estimator.labels_
bottomUp3rdLevelDict = { 'group0':[],
                        'group1':[],
                        'group2':[],
                        'group3':[],
                        'group4':[],
                        'group5':[],
                        'group6':[],
                        'group7':[]
                       }
bottomUp3rdLevelStats = { 'group0':0,
                        'group1':0,
                        'group2':0,
                        'group3':0,
                        'group4':0,
                        'group5':0,
                        'group6':0,
                        'group7':0
                       }
totalMisclassified8Clusters = 0
#get indices for each group and calcualte misclassification number and accuracy based on majority of case or ctrl in group
for label in np.unique(bottomUp8ClusterLabels):
    group  = 'group{0}'.format(label)
    bottomUp3rdLevelDict[group] = X.iloc[np.where(bottomUp8ClusterLabels==label)]
    tempAccuracy = sum(bottomUp3rdLevelDict[group].index<176)/len(bottomUp3rdLevelDict[group])
    clusterSize = len(bottomUp3rdLevelDict[group])
    if tempAccuracy>.5:
        bottomUp3rdLevelStats[group] = [clusterSize,sum(bottomUp3rdLevelDict[group].index>176), tempAccuracy]
    else:
        bottomUp3rdLevelStats[group] = [clusterSize,sum(bottomUp3rdLevelDict[group].index<176),1-tempAccuracy]
    totalMisclassified8Clusters = totalMisclassified8Clusters  + bottomUp3rdLevelStats[group][0]  

In [ ]:
bottomUp3rdLevelStats

In [ ]:
def new_euclidean_distances(X, Y=None, Y_norm_squared=None, squared=False): 
    return cosine_similarity(X,Y)

# monkey patch (ensure cosine dist function is used)
KMeans.euclidean_distances = new_euclidean_distances 

estimator = KMeans(n_clusters = 2)
estimator.fit(X)
max(sum(abs((estimator.labels_+1)- Y.as_matrix()))/len(Y),1-sum(abs((estimator.labels_+1)- Y.as_matrix()))/len(Y))
results = abs((estimator.labels_+1)- Y.as_matrix())
if sum(results)/len(results) > .5:
    misclassifiedTopDown = np.nonzero(results-1)
    pT = np.where(estimator.labels_ ==0)
    nT = np.where(estimator.labels_ ==1)
else:
    misclassifiedTopDown = np.nonzero((results))
    pT = np.where(estimator.labels_ ==1)
    nT = np.where(estimator.labels_ ==0)
patientList = []
for pat in patientLabels[misclassifiedTopDown[0]]:
    patientList.append(pat)
print("The accuracy is %f"%(1-len(misclassifiedTopDown[0])/364))
print(patientList)

In [ ]:
TPR = len(np.intersect1d(pT[0][pT[0]<176],P))/len(P)
FPR = 1 - len(np.intersect1d(pT[0][pT[0]<176],P))/len(P)
print("The true Positie rate is %f and the False Positive rate it %f"%(TPR,FPR) )
print("The size of the clusters are %d for the positve group and %d for the negative group"%(len(pT[0]),len(nT[0])))

In [ ]:
len(misclassifiedTopDown[0])

In [ ]:
import matplotlib_venn as v
from matplotlib import pyplot as plt
pBset = set(pB[0])
pTset = set(pT[0])
common = pBset & pTset
v.venn2([pBset,pTset],set_labels = ('Bottom Up','TopDown'))
plt.title('Comparison of Data Points in common classified as Case by Top Down and Bottom Up')
plt.show()

pBCorrectset = set(pB[0][pB[0]<176])
pTCorrectset = set(pT[0][pT[0]<176])
v.venn2([pBCorrectset,pTCorrectset],set_labels = ('Bottom Up','TopDown'))
plt.title('Comparison of Data Points in common correctly classified as Case by Bottom Up and Top Down')
plt.show()

pBIncorrectset = set(pB[0][pB[0]>175])
pTIncorrectset = set(pT[0][pT[0]>175])
v.venn2([pBIncorrectset,pTIncorrectset],set_labels = ('Bottom Up','TopDown'))
plt.title('Comparison of Data Points in common incorrectly classified as Case by Bottom Up and Top Down')
plt.show()

In [ ]:
len(np.intersect1d(pB[0][pB[0]<176],P))/len(P)

In [ ]:
#second iteration
group0 = np.where(estimator.labels_==0)[0]
group1 = np.where(estimator.labels_==1)[0]
group0X = X.iloc[group0]
group1X = X.iloc[group1]
estimator = KMeans(n_clusters = 2)
estimator.fit(group0X)
group0ClustersubsetLabels  = estimator.labels_
estimator = KMeans(n_clusters = 2)
estimator.fit(group1X)
group1ClustersubsetLabels  = estimator.labels_

In [ ]:
group0 = np.where(group0ClustersubsetLabels==0)[0]
group1 = np.where(group0ClustersubsetLabels==1)[0]
group2 = np.where(group1ClustersubsetLabels==0)[0]
group3 = np.where(group1ClustersubsetLabels==1)[0]

#split into four groups
group2level0X = group0X.iloc[group0]
group2level1X = group0X.iloc[group1]
group2level2X = group1X.iloc[group2]
group2level3X = group1X.iloc[group3]

topDown2ndLevelDict = {'group0':group2level0X,
                        'group1':group2level1X,
                        'group2':group2level2X,
                        'group3':group2level3X
                       }
topDown2ndLevelStats = {'group0':0,
                        'group1':0,
                        'group2':0,
                        'group3':0
                       }


for i in range(0,4):
    group = 'group{0}'.format(i)
    tempAccuracy = sum(topDown2ndLevelDict[group].index<176)/len(topDown2ndLevelDict[group])
    clusterSize = len(topDown2ndLevelDict[group])
    if tempAccuracy>.5:
        topDown2ndLevelStats[group] = [clusterSize, tempAccuracy]
    else:
        topDown2ndLevelStats[group] = [clusterSize,1-tempAccuracy]
topDown2ndLevelStats


In [ ]:
#estimators of the 4 nodes
estimator = KMeans(n_clusters = 2)
estimator.fit(group2level0X)
group0level2ClustersubsetLabels  = estimator.labels_
estimator = KMeans(n_clusters = 2)
estimator.fit(group2level1X)
group1level2ClustersubsetLabels  = estimator.labels_
estimator = KMeans(n_clusters = 2)
estimator.fit(group2level2X)
group2level2ClustersubsetLabels  = estimator.labels_
estimator = KMeans(n_clusters = 2)
estimator.fit(group2level3X)
group3level2ClustersubsetLabels  = estimator.labels_

group0 = np.where(group0level2ClustersubsetLabels==0)[0]
group1 = np.where(group0level2ClustersubsetLabels==1)[0]
group2 = np.where(group1level2ClustersubsetLabels==0)[0]
group3 = np.where(group1level2ClustersubsetLabels==1)[0]
group4 = np.where(group2level2ClustersubsetLabels==0)[0]
group5 = np.where(group2level2ClustersubsetLabels==1)[0]
group6 = np.where(group3level2ClustersubsetLabels==0)[0]
group7 = np.where(group3level2ClustersubsetLabels==1)[0]

group3level0X = group2level0X.iloc[group0]
group3level1X = group2level0X.iloc[group1]
group3level2X = group2level1X.iloc[group2]
group3level3X = group2level1X.iloc[group3]
group3level4X = group2level2X.iloc[group4]
group3level5X = group2level2X.iloc[group5]
group3level6X = group2level3X.iloc[group6]
group3level7X = group2level3X.iloc[group7]

topDown3rdLevelDict = { 'group0':group3level0X,
                        'group1':group3level1X,
                        'group2':group3level2X,
                        'group3':group3level3X,
                        'group4':group3level4X,
                        'group5':group3level5X,
                        'group6':group3level6X,
                        'group7':group3level7X
                       }
topDown3rdLevelStats = { 'group0':0,
                        'group1':0,
                        'group2':0,
                        'group3':0,
                        'group4':0,
                        'group5':0,
                        'group6':0,
                        'group7':0
                       }

for i in range(0,8):
    group = 'group{0}'.format(i)
    tempAccuracy = sum(topDown3rdLevelDict[group].index<176)/len(topDown3rdLevelDict[group])
    clusterSize = len(topDown3rdLevelDict[group])
    if tempAccuracy>.5:
        topDown3rdLevelStats[group] = [clusterSize,sum(topDown3rdLevelDict[group].index>176), tempAccuracy]
    else:
        topDown3rdLevelStats[group] = [clusterSize,sum(bottomUp3rdLevelDict[group].index<176),1-tempAccuracy]
topDown3rdLevelStats
